# 210312 Problem

My cake shop is so popular, I'm adding some tables and hiring wait staff so folks can have a cute sit-down cake-eating experience.

I have two registers: one for take-out orders, and the other for the other folks eating inside the cafe. All the customer orders get combined into one list for the kitchen, where they should be handled first-come, first-served.

Recently, some customers have been complaining that people who placed orders after them are getting their food first. Yikes—that's not good for business!

To investigate their claims, one afternoon I sat behind the registers with my laptop and recorded:

The take-out orders as they were entered into the system and given to the kitchen. (take_out_orders)
The dine-in orders as they were entered into the system and given to the kitchen. (dine_in_orders)
Each customer order (from either register) as it was finished by the kitchen. (served_orders)
Given all three lists, write a function to check that my service is first-come, first-served. All food should come out in the same order customers requested it.

We'll represent each customer order as a unique integer.

As an example,
```
 Take Out Orders: [1, 3, 5]
 Dine In Orders : [2, 4, 6]
 Served Orders  : [1, 2, 4, 6, 5, 3]
```
would not be first-come, first-served, since order 3 was requested before order 5 but order 5 was served first.

But,
```
 Take Out Orders: [17, 8, 24]
 Dine In Orders : [12, 19, 2]
 Served Orders  : [17, 8, 12, 19, 24, 2]
```
would be first-come, first-served.

Note: Order numbers are arbitrary. They do not have to be in increasing order.

## Observation

1. The order for take out and dine in shares the same queue - they won't have any overlaps
2. Both lists can be empty.
3. Served order needs to keep the order of respective orders, does not check the priority between the two lists.
4. At each point in time, I can always know the two "next possible order number" if I want to keep the order within each of the order lists.
5. Using pointer to track the possible values would be tricky because we need to check for validity of the data before every access to the array.
6. Served Orders is NOT necessarily going to be a union of the take out and dine in - there can be extra order that is not tracked in the system.

## Approach

Now we can't use pointers because that would require `not array` check before every access to the array, and nothing should ever be that cumbersome. With the intuition that I can know the next possible order numbers at all time, I can do the following:

1. write a helper function that will get possible order number at each given point. Make sure this function goes from the most specific case to the most general.
    - check if both lists are empty, in which case we have gone through all orders successfully
    - if one of the two lists are empty, return the first element of the other list
    - if not, this means we have value in both lists so return the first element of both lists
2. use the helper function to get the possible value
3. traverse the array and check if the element is in the possible order list. If not, we know this is not a FCFS order so return False
4. once we know the order is valid, check which array it is in. Then, pop the first element from that list using `popleft()`. we know for a fact that on the initial run. This pop() will guarantee that the possible order value will ALWAYS come from the first element in the list, thereby eliminating the need to use pointers at all.
5. With shortened list, re-calculate the possible order list
6. Once we come out of the loop, check to see if our possible order list have any value in it. If so, we know there's an order not tracked by the take out / dine in system so this is not a valid FCFS.

## Code

In [1]:
# Utility Function for getting the possible order number list
def getPossibleOrder(takeout: [int], dinein: [int]) -> [int]:
    # If both are empty
    if not takeout and not dinein:
        return []
    # If one of the two are empty
    elif not takeout:
        return [dinein[0]]
    elif not dinein:
        return [takeout[0]]
    # Both lists have value
    else:
        return [takeout[0], dinein[0]]

def is_first_come_first_served(take_out_orders: [int], dine_in_orders: [int], served_orders: [int]) -> bool:
    
    # get initial possible order list
    possibleOrder = getPossibleOrder(take_out_orders, dine_in_orders)
    
    # loop through all elements of the served order
    for i, n in enumerate(served_orders):
        # if current order is not in the possible order list, it's not valid FCFS
        if n not in possibleOrder:
            return False
        # if the order is valid, deque the fulfilled order from the proper list
        elif n in take_out_orders:
            take_out_orders.pop(0)
        else:
            dine_in_orders.pop(0)
        # recalculate the possible order
        possibleOrder = getPossibleOrder(take_out_orders, dine_in_orders)

    # check to see if any order was missed in the system
    return not possibleOrder

## Test

In [2]:
# Install with `pip install termcolor`
from termcolor import colored

def assertTest(result: bool) -> None:
    if result:
        print(colored("PASS", "green"))
    else:
        print(colored("FAIL", "red"))

assertTest(True  == is_first_come_first_served([1, 4, 5],    [2, 3, 6],   [1, 2, 3, 4, 5, 6]))
assertTest(False == is_first_come_first_served([1, 5],       [2, 3, 6],   [1, 2, 6, 3, 5]))
assertTest(True  == is_first_come_first_served([],           [2, 3, 6],   [2, 3, 6]))
assertTest(False == is_first_come_first_served([1, 5],       [2, 3, 6],   [1, 6, 3, 5]))
assertTest(False == is_first_come_first_served([1, 5],       [2, 3, 6],   [1, 2, 3, 5, 6, 8]))
assertTest(False == is_first_come_first_served([1, 9],       [7, 8],      [1, 7, 8]))
assertTest(False == is_first_come_first_served([55, 9],      [7, 8],      [1, 7, 8, 9]))
assertTest(True  == is_first_come_first_served([27, 12, 18], [55, 31, 8], [55, 31, 8, 27, 12, 18]))

PASS
PASS
PASS
PASS
PASS
PASS
PASS
PASS


## Complexity

### Time
`O(n)` because we are visiting every element in the `served_orders` exactly once.

### Space
`O(1)` because we are creating a list that will hold at most 2 elements, which means this does not grow with the size of the input.